## (a) Code for the MTZ formulation

# Mathematical Formulation of the Traveling Salesman Problem

## Decision Variables
- $x_{ij}$: Binary variable that equals 1 if the path from city $i$ to city $j$ is included in the tour, and 0 otherwise.
- $u_i$: Continuous variable representing the position of city $i$ in the tour to eliminate subtours.

## Parameters
- $d_{ij}$: Distance from city $i$ to city $j$.
- $n$: Number of cities.

## Objective Function
Minimize the total distance traveled:

$$
\text{Minimize} \quad Z = \sum_{i=1}^{n} \sum_{j=1, j \neq i}^{n} d_{ij} x_{ij}
$$

## Constraints

### Arrival Constraint
Each city $j$ must be arrived at exactly once.

$$
\sum_{i=1, i \neq j}^{n} x_{ij} = 1 \quad \forall j \in \{1, \ldots, n\}
$$

### Departure Constraint
Each city $i$ must be departed from exactly once.

$$
\sum_{j=1, j \neq i}^{n} x_{ij} = 1 \quad \forall i \in \{1, \ldots, n\}
$$

### Subtour Elimination Constraints (MTZ)
These constraints prevent the formation of subtours by ensuring that each city (except the first one, arbitrarily chosen) is visited in a specific order.

$$
u_i - u_j + nx_{ij} \leq n-1 \quad \forall i, j \in \{2, \ldots, n\}, i \neq j
$$

## Model Summary
The TSP model seeks to:
- Minimize the total distance of the tour.
- Ensure each city is visited exactly once and eliminate subtours.


In [3]:
import gurobipy as gp
from gurobipy import GRB
import re, time

def read_distance_matrix(filename):
    with open(filename, 'r') as f:
        dist = []
        for line in f:
            l_ints = re.findall(r'\d+\.\d+', line)
            l_ints = [int(float(value)) for value in l_ints]
            dist.append(l_ints)
    return dist

def find_next_city(current_city, xij):
    """Find the next city in the tour from the current city."""
    for j, visited in enumerate(xij[current_city]):
        if visited > 0.5:  # Assuming binary decision variables might not be exactly 1 due to numerical precision
            return j
    return None

def extract_optimal_route(xij):
    """Extract the optimal route from the TSP solution."""
    n_cities = len(xij)
    route = [0]  # Start from the first city (index 0)
    current_city = 0

    while True:
        next_city = find_next_city(current_city, xij)
        if next_city is None or next_city == 0:
            break  # End the tour if no next city is found or if we have returned to the starting city
        route.append(next_city)
        current_city = next_city

        if len(route) > n_cities:
            print("Loop detected in the route. Please check the solution.")
            break

    return route

start_time = time.time()

# Read the distance matrix and number of cities
dist_matrix = read_distance_matrix("TSP_instance.txt")
n_cities = len(dist_matrix)

# Initialize the TSP model
tsp_model = gp.Model("tsp")

# Add variables
xij = [[tsp_model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}") for j in range(n_cities)] for i in range(n_cities)]
ui = [tsp_model.addVar(vtype=GRB.CONTINUOUS, name=f"u_{i}") for i in range(n_cities)]

# Add constraints
# Each city is arrived at exactly once
for j in range(n_cities):
    tsp_model.addConstr(gp.quicksum(xij[i][j] for i in range(n_cities) if i != j) == 1, name=f"c1_{j}")

# Each city is left exactly once
for i in range(n_cities):
    tsp_model.addConstr(gp.quicksum(xij[i][j] for j in range(n_cities) if i != j) == 1, name=f"c2_{i}")

# Subtour elimination constraints (MTZ)
for i in range(1, n_cities):  # Start from 1 to skip the first city
    for j in range(1, n_cities):  # Start from 1 as the first city is considered the starting point
        if i != j:
            tsp_model.addConstr(ui[i] - ui[j] + n_cities * xij[i][j] <= n_cities - 1, name=f"MTZ_{i}_{j}")

# Set the objective function
tsp_model.setObjective(gp.quicksum(xij[i][j] * dist_matrix[i][j] for i in range(n_cities) for j in range(n_cities) if i != j), GRB.MINIMIZE)

# Optimize the model
tsp_model.optimize()

# Check and print the solution
if tsp_model.status == GRB.Status.OPTIMAL:
    solution = [[int(xij[i][j].X) for j in range(n_cities)] for i in range(n_cities)]
else:
    print("No optimal solution found.")
    
end_time = time.time()

# Calculate the total time taken
total_time = end_time - start_time
print(f"Total time taken to find the solution: {total_time:.2f} seconds")

optimal_route = extract_optimal_route(solution)

print("Optimal Route:", "->".join(map(str, [city + 1 for city in optimal_route])))  # +1 to adjust city indices for human-readable format (1-based indexing)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2552 rows, 2652 columns and 12450 nonzeros
Model fingerprint: 0xac5feb1d
Variable types: 51 continuous, 2601 integer (2601 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 0 rows and 52 columns
Presolve time: 0.03s
Presolved: 2552 rows, 2600 columns, 12450 nonzeros
Variable types: 50 continuous, 2550 integer (2550 binary)

Root relaxation: objective 3.775098e+02, 196 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  377.50980    0   

# (b) Iterative Solution for the Traveling Salesman Problem with Subtour Elimination

The following outlines an approach to solve the TSP by iteratively adding constraints to eliminate subtours until an optimal tour is found:

## Subtour Detection Function
A function `find_subtours` is defined to detect subtours within a given solution. It iterates through the solution, tracking visited cities to identify closed loops (subtours). Once a subtour is completed, it moves to the next set of unvisited cities until all cities are accounted for.

## Model Initialization
- The distance matrix is read, and the number of cities (`n_cities`) is determined.
- A TSP model is initialized using Gurobi, and binary variables (`xij`) are added to represent paths between cities.

## Constraints
- Constraints are added to ensure each city is arrived at and left exactly once.

## Objective Function
- The objective function is set to minimize the total distance of the tour.

## Iterative Solution
- The model is solved iteratively. After each optimization, `find_subtours` is called to detect any existing subtours.
- If a single subtour containing all cities is found, the optimal tour is identified, and the iteration stops.
- If multiple subtours are detected, additional subtour elimination constraints are added to the model to prevent these subtours in the next iteration.

## Results
- The total time taken to find the solution and the optimal route are printed. The optimal route is adjusted for 1-based indexing to enhance readability.

This iterative approach ensures that the model converges to a solution where a single tour visits all cities exactly once, effectively solving the TSP.


In [ ]:
# Detect subtours in the solution
def find_subtours(solution):
    unvisited = list(range(len(solution)))
    subtours = []
    while unvisited:
        subtour = [unvisited[0]]
        while True:
            current_city = subtour[-1]
            next_city = solution[current_city].index(1)
            if next_city in subtour:
                subtours.append(subtour)
                break
            subtour.append(next_city)
        unvisited = [j for j in unvisited if j not in subtour]
    return subtours

start_time = time.time()

# Read distances and determine the number of cities
dist = read_distance_matrix("TSP_instance.txt")
n_cities = len(dist)

# Initialize the TSP model
tsp_model = gp.Model("tsp")

# Add binary variables for paths between cities
xij = [[tsp_model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}") for j in range(n_cities)] for i in range(n_cities)]

# Add constraints for each city to be arrived at and left exactly once
for i in range(n_cities):
    tsp_model.addConstr(gp.quicksum(xij[i][j] for j in range(n_cities) if i != j) == 1, f"c1_{i}")
    tsp_model.addConstr(gp.quicksum(xij[j][i] for j in range(n_cities) if i != j) == 1, f"c2_{i}")

# Set the objective function to minimize the total distance
tsp_model.setObjective(gp.quicksum(xij[i][j] * dist[i][j] for i in range(n_cities) for j in range(n_cities) if i != j), GRB.MINIMIZE)

# Solve the model iteratively to eliminate subtours
iteration = 0
while True:
    tsp_model.optimize()
    solution = [[int(xij[i][j].X) for j in range(n_cities)] for i in range(n_cities)]
    subtours = find_subtours(solution)
    if len(subtours) == 1 and len(subtours[0]) == n_cities:
        print(f"Optimal tour found in iteration {iteration}")
        break
    else:
        print(f"Subtour(s) detected in iteration {iteration}. Adding constraints to eliminate. Subtours: {subtours}")
        for subtour in subtours:
            if len(subtour) < n_cities:
                tsp_model.addConstr(gp.quicksum(xij[i][j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)
                
    iteration += 1
    
end_time = time.time()

print('\n')
print("SUMMARY OF THE RESULT")
print('\n')

# Calculate the total time taken
total_time = end_time - start_time
print(f"Total time taken to find the solution: {total_time:.2f} seconds")


# Print the optimal route
optimal_route = [0]
for _ in range(n_cities - 1):
    next_city = solution[optimal_route[-1]].index(1)
    optimal_route.append(next_city)

print("Optimal Route:", "->".join(map(str, [city + 1 for city in optimal_route])))  # +1 for 1-based indexing
